In [ ]:
//String to Integer:

LOAD CSV  WITH HEADERS FROM 'https://data.neo4j.com/intro/movies/people.csv' AS row RETURN row.name, toInteger(row.born)  LIMIT 5;
LOAD CSV  WITH HEADERS FROM 'https://data.neo4j.com/intro/movies/movies.csv'  AS row RETURN row.title, row.tagline, toInteger(row.released) LIMIT 5;

//array:

LOAD CSV WITH HEADERS FROM 'https://data.neo4j.com/intro/movies/actors.csv' AS row
WITH row WHERE row.person = 'Hugo Weaving' RETURN split(coalesce(row.roles,""),";");

//persons:

LOAD CSV WITH HEADERS FROM 'https://data.neo4j.com/intro/movies/people.csv'  AS row
CREATE (p:Person)
SET p.name = row.name, p.born = toInteger(row.born);

//movies:

LOAD CSV WITH HEADERS FROM 'https://data.neo4j.com/intro/movies/movies.csv'  AS row
CREATE (m:Movie)
SET m.title = row.title, m.released = toInteger(row.released), m.tagline = row.tagline;

//Movie Index:

CREATE CONSTRAINT UniqueMovieTitleConstraint FOR (m:Movie) REQUIRE m.title IS UNIQUE;

//Person Index:

CREATE CONSTRAINT UniquePersonNameConstraint FOR (p:Person) REQUIRE p.name IS UNIQUE;

//Directors:

LOAD CSV WITH HEADERS FROM 'https://data.neo4j.com/intro/movies/directors.csv' AS row
MATCH (m:Movie {title: row.movie})
MERGE (p:Person {name: row.person})
WITH m,p
MERGE (p)-[:DIRECTED]->(m);


//acters:

LOAD CSV WITH HEADERS FROM 'https://data.neo4j.com/intro/movies/actors.csv'  as row
MATCH (p:Person) where p.name = row.person
MATCH (m:Movie) where m.title = row.movie
WITH row, p, m
MERGE (p)-[r:ACTED_IN]->(m)
SET r.roles = split(coalesce(row.roles,""),";");


 
